In [1]:
import cv2
import numpy as np
import os

def apply_gaussian_blur(frame, kernel_size=(7, 7)):
    return cv2.GaussianBlur(frame, kernel_size, 0)

def add_gaussian_noise(frame, mean=0, var=0.65):
    sigma = var ** 2
    gauss = np.random.normal(mean, sigma, frame.shape).astype('uint8')
    noisy_frame = cv2.add(frame, gauss)
    return noisy_frame


def process_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    #                                                                                           
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # 使用MP4编码器并保持原视频的帧率
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        blurred_frame = apply_gaussian_blur(frame)
        noisy_frame = add_gaussian_noise(blurred_frame)

        out.write(noisy_frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [2]:
from PIL import Image, ImageFilter, ImageEnhance
import numpy as np

def apply_gaussian_blur_gif(frame, radius=5):
    return frame.filter(ImageFilter.GaussianBlur(radius=radius))

def add_gaussian_noise_gif(frame, mean=0, var=5):
    np_frame = np.array(frame)
    sigma = var **  0.1
    gauss = np.random.normal(mean, sigma, np_frame.shape).astype('uint8')
    noisy_frame = cv2.add(np_frame, gauss)
    noisy_frame = np.clip(noisy_frame, 0, 255).astype('uint8')
    return Image.fromarray(noisy_frame)

def process_gif(input_path, output_path):
    gif = Image.open(input_path)
    frames = []
    try:
        while True:
            frame = gif.copy().convert('RGB')
            blurred_frame = apply_gaussian_blur_gif(frame)
            noisy_frame = add_gaussian_noise_gif(blurred_frame)
            frames.append(noisy_frame)  # Convert back to 'P' mode for GIF
            gif.seek(gif.tell() + 1)
    except EOFError:
        pass

    frames[0].save(output_path, save_all=True, append_images=frames[1:], loop=0)

In [6]:
from tqdm import tqdm
output_dir = r'D:\Astudying\VideoEval\distorted_overall_consistency\overall_consistency'
input_dir = r'D:\Astudying\VideoEval\data4dimensions\overall_consistency'
model_name_list = ['cogvideox5b','kling', 'gen3','videocrafter2', 'pika', 'show1', 'lavie']
for model_name in model_name_list:
    model_video_dir = os.path.join(input_dir, model_name)
    for video_name in tqdm(os.listdir(model_video_dir)):
        if video_name in os.listdir(os.path.join(output_dir, model_name)):
            continue
        if video_name.split('_')[-1][0] == '0':
            video_path = os.path.join(model_video_dir, video_name)
            output_path = os.path.join(output_dir, model_name, video_name)
            process_video(video_path, output_path)
    print('finish',model_name)

100%|██████████| 273/273 [00:39<00:00,  6.98it/s]


finish cogvideox5b


100%|██████████| 273/273 [05:27<00:00,  1.20s/it]


finish kling


100%|██████████| 273/273 [13:11<00:00,  2.90s/it]


...

finish pika


100%|██████████| 274/274 [00:13<00:00, 20.98it/s]


finish show1


100%|██████████| 273/273 [00:10<00:00, 26.12it/s]

finish lavie
